# This notebook test all direct spectral estimator of the structure factor:
Scattering intensity, taper, debiased taper, multitaper, debiased multitaper

In [ ]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib as mpl
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'

In [ ]:
import numpy as np
from structure_factor.data import load_data
import structure_factor.utils as utils
import structure_factor.spectral_estimator as spe
from structure_factor.structure_factor import StructureFactor
# restrict the Ginibre pattern to a cubic window
from structure_factor.spatial_windows import BoxWindow, BallWindow
from structure_factor.point_pattern import PointPattern
import structure_factor.tapers as tapers

In [ ]:
params.setdefault("step_size", 0.1)

# Test direct spectral estimator for Ginibre ensemble On allowed values 

In [ ]:
#ginibre 
ginibre_pp = load_data.load_ginibre()
#restrict to box window
L = ginibre_pp.window.radius/np.sqrt(2) # side length of the cubic window

bounds = [[-L/2, L/2], [-L/2, L/2]] 
window = BoxWindow(bounds) # create a cubic window
ginibre_pp_box = ginibre_pp.restrict_to_window(window)
#ginibre points
points = ginibre_pp_box.points
sf_ginibre = StructureFactor(ginibre_pp_box)

## Scattering inetnsity

In [ ]:
# on allowed values 
allowed_k, si = sf_ginibre.scattering_intensity()

In [ ]:
x = np.linspace(-8, 8, 200)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
# on k 
k, si1 = sf_ginibre.scattering_intensity(k=k, debiased=False)
# on k debiased directly
k, si2 = sf_ginibre.scattering_intensity(k=k, debiased=True)
# on k debiased undirectly
k, si3 = sf_ginibre.scattering_intensity(k=k, debiased=True, undirect=True)

In [ ]:
si.shape

In [ ]:
print("Scattering intensity")
k_norm = np.linalg.norm(allowed_k, axis=1)
fig, axis = plt.subplots(figsize=(6,6))
fig = sf_ginibre.plot_scattering_intensity(allowed_k, si, axes=axis, plot_type="radial",exact_sf=utils.structure_factor_ginibre(k_norm),
                                            bins=40, error_bar=True, file_name="si_allowed_k")

In [ ]:

shape_mesh = (150, 150)
allowed_k, K = utils.allowed_wave_vectors(d=2, L=L, k_max=10, meshgrid_shape=shape_mesh)


k_norm_1, si_1 = sf_ginibre.scattering_intensity(k_max=10, meshgrid_shape=shape_mesh) # on allowed values
_, i0_1 = sf_ginibre.scattering_intensity_debiased(allowed_k)
_, i0_2 = sf_ginibre.scattering_intensity_debiased(allowed_k)

k_nomr_3, sf_mtpp = sf_ginibre.multitaper_periodogram(P=2, k= allowed_k)
_, sf_mtpp_debiased = sf_ginibre.multitaper_periodogram_debiased(P=3, k=allowed_k, undirect=False)
_, sf_mtpp_debiased_undirect = sf_ginibre.multitaper_periodogram_debiased(P=3, k=allowed_k, undirect=True)


In [ ]:
#plot the result
#plot result
print("Scattering intensity on allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, si_1, plot_type="all",exact_sf=utils.structure_factor_ginibre,
                                            bins=40, error_bar=True)


In [ ]:
print("debiased estimator on allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, i0_1, plot_type="all",exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("Second debiased estimator on allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, i0_2, plot_type="all",exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("multitaper on allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1,sf_mtpp, plot_type="all",
                                              exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True,
                                              #file_name="si_ginibre.pdf"
                                              )

In [ ]:
print("debiased multitaper on allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1,sf_mtpp_debiased, plot_type="all",
                                              exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("undirect debiased multitatper")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1,sf_mtpp_debiased_undirect, plot_type="all",
                                              exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

## On non allowed values

In [ ]:
# k gride
x = np.linspace(-8, 8, 150)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
sf_ginibre = StructureFactor(ginibre_pp_box)

In [ ]:
k_norm_1, si_1 = sf_ginibre.scattering_intensity(k) # on allowed values
_, i0_1 = sf_ginibre.scattering_intensity_debiased(k)
_, i0_2 = sf_ginibre.scattering_intensity_debiased(k, undirect=True)

k_norm_3, sf_mtpp = sf_ginibre.multitaper_periodogram(P=2, k= k)
_, sf_mtpp_debiased = sf_ginibre.multitaper_periodogram_debiased(P=2, k=k, undirect=False)
_, sf_mtpp_debiased_undirect = sf_ginibre.multitaper_periodogram_debiased(P=2, k=k, undirect=True)


In [ ]:
print("exact periodogram")
periodogram = utils.structure_factor_ginibre(k_norm_1)
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, periodogram, plot_type="all")

In [ ]:
print("Scattering intensity on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, si_1, plot_type="all", exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("Direct debiased estimator on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, i0_1, plot_type="all",exact_sf=utils.structure_factor_ginibre,
                                              bins=50, error_bar=True)

In [ ]:
print("undirect debiased estimator on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, i0_2, plot_type="all", exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("multitaper on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1,sf_mtpp, plot_type="all",exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("debiased multitaper on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1,sf_mtpp_debiased, plot_type="all",exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("undirect debiased multitatper on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1,sf_mtpp_debiased_undirect, plot_type="all", exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("absolut value of the undirect debiased multitatper on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1,np.abs(sf_mtpp_debiased_undirect), plot_type="all", exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

# Isotropic estimator

In [ ]:
import time
import structure_factor.isotropic_njit as iso_numba

In [ ]:
#ginibre 
ginibre_pp = load_data.load_ginibre()
#restrict to small ball window
R = 100
center = [0,0] 
window = BallWindow(radius=R, center=center) # create a cubic window
ginibre_pp_ball = ginibre_pp.restrict_to_window(window)
#ginibre points
points = ginibre_pp_ball.points
sf_ginibre = StructureFactor(ginibre_pp_ball)

k = np.linspace(1, 10, 16**2)

In [ ]:
from structure_factor.utils import  bessel1_zeros
x = bessel1_zeros(1, 200)
k = x/ginibre_pp_ball.window.radius

In [ ]:
ginibre_pp.window.radius

In [ ]:
from structure_factor.spectral_estimator import isotropic_bartlett_estimator

%time k_norm, s_ib = isotropic_bartlett_estimator(k, ginibre_pp_ball)


In [ ]:
print("Bartlett isotropic estimator on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k, s_ib, plot_type="radial", exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)


In [ ]:

%time k_norm, s_ib = iso_numba.isotropic_estimator(ginibre_pp_ball)

In [ ]:
print("Bartlett isotropic estimator on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm[1:], s_ib[1:], plot_type="radial", exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
np.min(k_norm)

In [ ]:
k_norm[0]

In [ ]:
2*np.pi/70

In [ ]:
k_ = np.array([[1, 0], [2, 0], [4, 1], [5, 2]])
K = np.atleast_2d(k_)
norm_k = np.linalg.norm(k_, axis=1)
norm_k

## For poisson

In [ ]:
pois_pp = load_data.load_poisson()
#restrict to box window
L = 70 # side length of the cubic window
bounds = [[-L/2, L/2], [-L/2, L/2]] 
window = BoxWindow(bounds) # create a cubic window
pois_pp_box = pois_pp.restrict_to_window(window)

In [ ]:
pois_pp_box.params


In [ ]:
# k gride
x = np.linspace(-8, 8, 100)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
sf_pois = StructureFactor(pois_pp_box)

k_norm_1, si_1 = sf_pois.scattering_intensity(k) # on allowed values
_, i0_1 = sf_pois.scattering_intensity_debiased(k)
_, i0_2 = sf_pois.scattering_intensity_debiased(k, undirect=True)

_, sf_mtpp = sf_pois.multitaper_periodogram(P=3, k= k)
_, sf_mtpp_debiased = sf_pois.multitaper_periodogram_debiased(P=3, k=k, undirect=False)
_, sf_mtpp_debiased_undirect = sf_pois.multitaper_periodogram_debiased(P=3, k=k, undirect=True)


In [ ]:
print("exact periodogram")
periodogram = utils.structure_factor_poisson(k_norm_1)
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, periodogram, plot_type="all")

In [ ]:
print("Scattering intensity on non allowed values")
fig = sf_pois.plot_scattering_intensity(k_norm_1, si_1, plot_type="all",exact_sf=utils.structure_factor_poisson,
                                            bins=40, error_bar=True)


In [ ]:
print(" Direct debias Scattering intensity on non allowed values")
fig = sf_pois.plot_scattering_intensity(k_norm_1, i0_1, plot_type="all",exact_sf=utils.structure_factor_poisson,
                                            bins=40, error_bar=True)


In [ ]:
print("undirect debias Scattering intensity on non allowed values")
fig = sf_pois.plot_scattering_intensity(k_norm_1, i0_2, plot_type="all",exact_sf=utils.structure_factor_poisson,
                                            bins=40, error_bar=True)


In [ ]:
print("multitaper on no allowed values")
fig = sf_pois.plot_scattering_intensity(k_norm_1, sf_mtpp, plot_type="all",exact_sf=utils.structure_factor_poisson,
                                            bins=40, error_bar=True)


In [ ]:
print("Direct debiased multitaper on no allowed values")
fig = sf_pois.plot_scattering_intensity(k_norm_1, sf_mtpp_debiased, plot_type="all",exact_sf=utils.structure_factor_poisson,
                                            bins=40, error_bar=True)

In [ ]:
print("Undirect debiasd multitaper on allowed values")
fig = sf_pois.plot_scattering_intensity(k_norm_1, sf_mtpp_debiased_undirect, plot_type="all",exact_sf=utils.structure_factor_poisson,
                                            bins=40, error_bar=True)

In [ ]:
pois_pp = load_data.load_poisson()
pois_pp.window.bounds

# Thomas point process

In [ ]:


# Simulation window parameters
L = 80
xMin = -L;
xMax = L;
yMin = -L;
yMax = L;

# Parameters for the parent and daughter point processes
lambdaParent = 1/(20*np.pi);  # density of parent Poisson point process
lambdaDaughter = 20*np.pi;  # mean number of points in each cluster
sigma = 2;  # sigma for normal variables (ie random locations) of daughters

# Extended simulation windows parameters
rExt=6*sigma; # extension parameter 
# for rExt, use factor of deviation sigma eg 5 or 6
xMinExt = xMin - rExt;
xMaxExt = xMax + rExt;
yMinExt = yMin - rExt;
yMaxExt = yMax + rExt;
# rectangle dimensions
xDeltaExt = xMaxExt - xMinExt;
yDeltaExt = yMaxExt - yMinExt;
areaTotalExt = xDeltaExt * yDeltaExt;  # area of extended rectangle

# Simulate Poisson point process for the parents
numbPointsParent = np.random.poisson(areaTotalExt * lambdaParent);# Poisson number of points
# x and y coordinates of Poisson points for the parent
xxParent = xMinExt + xDeltaExt * np.random.uniform(0, 1, numbPointsParent);
yyParent = yMinExt + yDeltaExt * np.random.uniform(0, 1, numbPointsParent);

# Simulate Poisson point process for the daughters (ie final poiint process)
numbPointsDaughter = np.random.poisson(lambdaDaughter, numbPointsParent);
numbPoints = sum(numbPointsDaughter);  # total number of points

# Generate the (relative) locations in Cartesian coordinates by
# simulating independent normal variables
xx0 = np.random.normal(0, sigma, numbPoints);  # (relative) x coordinaets
yy0 = np.random.normal(0, sigma, numbPoints);  # (relative) y coordinates

# replicate parent points (ie centres of disks/clusters)
xx = np.repeat(xxParent, numbPointsDaughter);
yy = np.repeat(yyParent, numbPointsDaughter);

# translate points (ie parents points are the centres of cluster disks)
xx = xx + xx0;
yy = yy + yy0;

# thin points if outside the simulation window
booleInside = ((xx >= xMin) & (xx <= xMax) & (yy >= yMin) & (yy <= yMax));
# retain points inside simulation window
xx = xx[booleInside];  
yy = yy[booleInside];

# Plotting
plt.scatter(xx, yy, edgecolor='b', facecolor='none', alpha=0.5, s=2);
plt.xlabel("x");
plt.ylabel("y");
plt.axis('equal');

In [ ]:
thomas_points = np.column_stack((xx , yy))
bounds = [[xMin, xMax], [yMin, yMax]]
thomas_window = BoxWindow(bounds)
thomas_intensity = lambdaParent*lambdaDaughter
intensity_parent = lambdaParent
sigma = sigma
thomas_pp = PointPattern(thomas_points, thomas_window )


In [ ]:
r = np.linspace(0, 30, 200)
pcf_ths = pcf_thomas(r)
d=2
plt.plot(r, utils.pair_correlation_function_thomas(r, d, lambdaParent, sigma ), 'b')
plt.plot(r, utils.structure_factor_thomas(r, d=d, rho_child=lambdaDaughter, sigma=sigma), 'k')
#plt.plot(k_norm, si, 'g.')
plt.plot(r, np.ones_like(pcf_ths), 'r')

In [ ]:
sf_thomas_box = StructureFactor(thomas_pp)

# pcf

In [ ]:
sf_thomas = StructureFactor(thomas_pp)
# estimate the pair correlation function using pcf.fv
pcf_fv = sf_thomas.compute_pcf(method="fv", Kest=dict(rmax=15),
                                        fv=dict(method="b", spar=0.1))
pcf_fv

In [ ]:
pcf_th = utils.pair_correlation_function_thomas(pcf_fv["r"], d=2, rho_parent=lambdaParent,sigma=sigma)
fig = sf_thomas_box.plot_pcf(pcf_fv,
                          exact_pcf=pcf_th,
                          figsize=(10,6),
                          color=['b', 'grey', 'darkcyan'],
                          style=[".", "o", "^"]
                          )

## SI

In [ ]:
shape_mesh = (100, 100)
k, _ = utils.allowed_wave_vectors(d=2, L=L_res, k_max=2, meshgrid_shape=shape_mesh)

In [ ]:
# k gride
x = np.linspace(-5, 5, 111)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))

In [ ]:

k_norm, si = sf_thomas_box.scattering_intensity(k)

In [ ]:
print("Scattering intensity on non allowed values")
exact_sf = utils.structure_factor_thomas(k_norm, rho_child=lambdaDaughter, d=2, sigma=2)
fig = sf_thomas_box.plot_scattering_intensity(k_norm, si, plot_type="all", exact_sf=exact_sf,
                                              bins=40, error_bar=True)

In [ ]:
k_norm2, sf_mtpp_debiased = sf_thomas.multitaper_periodogram_debiased(P=2, k=k, undirect=False)


In [ ]:
k_norm2.shape

In [ ]:
print("MTPP debiased directly on non allowed values")
exact_sf = utils.structure_factor_thomas(k_norm2, rho_child=lambdaDaughter, d=2, sigma=2)
fig = sf_thomas_box.plot_scattering_intensity(k_norm2, sf_mtpp_debiased, plot_type="all", exact_sf=exact_sf,
                                              bins=40, error_bar=True)

In [ ]:
pcf_th = utils.pair_correlation_function_thomas(pcf_fv["r"], d=2, rho_parent=lambdaParent,sigma=sigma)
fig = sf_thomas_box.plot_pcf(pcf_fv,
                          exact_pcf=pcf_th,
                          figsize=(10,6),
                          color=['b', 'grey', 'darkcyan'],
                          style=[".", "o", "^"]
                          )

In [ ]:
thomas_points = np.column_stack((xx , yy))
bounds = [[xMin, xMax], [yMin, yMax]]
thomas_window = BoxWindow(bounds)
thomas_intensity = lambdaParent*lambdaDaughter
intensity_parent = lambdaParent
sigma = sigma
params = {"intensity_parent":intensity_parent, "sigma":sigma }
thomas_pp = PointPattern(thomas_points, thomas_window, thomas_intensity, **params )
thomas_pp.plot(window_res=BoxWindow([[-20, 20], [-20, 20]]))

import pickle
thomas_pp = {"points":thomas_points, "bounds":bounds, "intensity":thomas_intensity, "intensity_parent":intensity_parent, "sigma":sigma}
with open('thomas_data.pkl', 'wb') as f:
    pickle.dump(thomas_pp, f)